In [ ]:
# Unsloth설치
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-zap0x8md/unsloth_5996794309a245f4b7f2ad0c97aa989d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-zap0x8md/unsloth_5996794309a245f4b7f2ad0c97aa989d
  Resolved https://github.com/unslothai/unsloth.git to commit 600ffe2a175312e7f2fbcdffbe6b99e3df20c417
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#파인튜닝 모델 로드
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    #위 모델에서 4bit모델이 T4에서 훈련이 가능하므로 호출
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
#명령어 구조
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}
### Input:
{}

### Response:
{}
"""

#End Of Sentence토큰의 끝을 알려주는 토큰 FILE의 EOF같은 것
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
#데이터셋 불러오기
dataset = load_dataset("jojo0217/korean_safe_conversation", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/26979 [00:00<?, ? examples/s]

Map:   0%|          | 0/26979 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

#파인튜닝을 위한 트레이너 셋팅
trainer = SFTTrainer(
    #타겟 모델
    model = model,
    #변환용 토크나이저
    tokenizer = tokenizer,
    #데이터 셋
    train_dataset = dataset,
    #데이터셋에서 하나의 레코드의 딕셔너리의 필드네임
    dataset_text_field = "text",
    #최대 토큰수
    max_seq_length = max_seq_length,
    #전처리 프로세스 수
    dataset_num_proc = 2,
    # 동시 처리용 - 현재 쓸모 없음
    packing = False,
    # 트레이닝 인자
    args = TrainingArguments(
        #GPU마다의 배치크기
        per_device_train_batch_size = 8,
        # 배치크기 * gradient_accumulation_steps = 32개씩 학습
        gradient_accumulation_steps = 4,
        #(learning_tate / warmup_steps) * step = 현재 스탭의 학습률
        warmup_steps = 5,
        # 데이터셋을 몇번 학습할건지 epoch
        num_train_epochs = 1,
        #최대 몇 스탭까지 공부할 것인가?
        max_steps = 200,
        #학습률
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        #몇 스탭마다 기록할지.
        logging_steps = 1,
        #최적화
        optim = "adamw_8bit",
        weight_decay = 0.01,
        #학습방법 현 선형적
        lr_scheduler_type = "linear",
        #랜덤성의 seed
        seed = 3407,
        #저장 위치
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/26979 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# cuda 장치 상태 획득
gpu_stats = torch.cuda.get_device_properties(0)
#사용중인 메모리 획득
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
#GPU 최대 메모리
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [ ]:
#파인튜닝 시작
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 26,979 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 1,024
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.261600
2,2.325500
3,2.204600
4,2.181000
5,2.009200
6,1.978300
7,1.859100
8,1.817900
9,1.688200
10,1.723700


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.73 GiB. GPU 

In [ ]:
#결과 출력
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
#alpaca_prompt = 학습시 만들어둔 Instruct 구조
#한번에 입력
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "당신은 지식이 풍부하고 공감 능력이 뛰어난 진로 상담사입니다. 당신의 역할은 경력 경로, 구직 활동, 또는 전문성 개발에 대한 조언을 구하는 개인들에게 안내와 지원, 그리고 정보를 제공하는 것입니다. 당신의 전문성을 활용하여 제공된 입력을 바탕으로 자세한 맞춤형 조언을 한국어로 제공하세요.", # instruction
        "커리어 관련하여 당신과 상담을 하고 싶습니다. 저는 컴퓨터를 임베디드, 인공지능, 백엔드, 프론트를 모두 아우르는 경력을 가졌지만 어디에 취업해야 할지 모르겠습니다.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
#alpaca_prompt = 학습시 만들어둔 Instruct 구조
#스트림 입력처리
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "재취업을 준비하는 50대분들에게 조언을 해주세요.", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

In [ ]:
import shutil
import os

quantization_methods = ["f16", "q8_0", "q4_k_m"]

for method in quantization_methods:
    # Save model to a temporary directory
    output_dir = f"model_{method}"

    # 양자화된 모델을 저장
    model.save_pretrained_gguf(
        output_dir,
        tokenizer,
        quantization_method=method
    )

    # Hugging Face Hub에 모델 업로드
    model.push_to_hub_gguf(
        "steve15963/FineTune_LLama3.1_4Bit_Use_jojo0217_korean_safe_conversation_and_unsloth",
        tokenizer,
        quantization_method=method,
        token="",
        repo_path_or_name=output_dir,  # 각 버전의 디렉토리를 지정
    )

    print(f"Uploaded {method} version to the same repository.")

    # 로컬에 저장된 모델 데이터 삭제
    shutil.rmtree(output_dir)
    print(f"Deleted local files for {method} version.")
